In [1]:
import sys
sys.path.append('../')
from config_sing import *

You are on Duke Server: Singularity: Proj CombEffect
BASE DIRECTORY:     /mount/work
PATH OF SOURCE:     /mount/work/source
PATH OF EXECUTABLE: /mount/work/exe
PATH OF ANNOTATION: /mount/work/annotation
PATH OF PROJECT:    /mount/project
PATH OF RESULTS:    /mount/work/out/proj_combeffect


In [2]:
import sqlite3
import itertools as it

In [3]:
fdiry = os.path.join(FD_RES, "nuc")
print(os.listdir(fdiry))

['Input5_20x', 'Input1_20x', 'TFX2_DMSO', 'Input3', 'TFX5_Dex', 'Input4_20x', 'TFX2_Dex', 'TFX5_DMSO', 'TFX4_Dex', 'Input4', 'Input5', 'Input1', 'Input2', 'TFX3_Dex', 'TFX4_DMSO', 'Input2_20x', 'TFX3_DMSO', 'Input3_20x']


In [4]:
fdiry = os.path.join(FD_RES, "nuc", "Input1_20x")
os.listdir(fdiry)

['target_PER1.bed.gz']

```
Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input1/target_PER1.bed.gz
#1_usercol	2_usercol	3_usercol	4_usercol	5_pct_at	6_pct_gc	7_num_A	8_num_C	9_num_G	10_num_T	11_num_N	12_num_oth	13_seq_len
chr17	8148117	8149012	1	0.423464	0.576536	196	268	248	183	0	0	895
chr17	8148122	8149107	1	0.428426	0.571574	219	290	273	203	0	0	985
```

In [5]:
fdiry = os.path.join(FD_RES, "nuc", "Input1_20x")
fname = "target_PER1.bed.gz"
fpath = os.path.join(fdiry, fname)

n_lines = 5
with gzip.open(fpath, "rb") as file:
    header = file.readline().decode('ASCII').strip().split('\t')
    print(header)
    
    lines  = it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')  
        print(lst)
        #print(prep_line(lst))

['#1_usercol', '2_usercol', '3_usercol', '4_usercol', '5_pct_at', '6_pct_gc', '7_num_A', '8_num_C', '9_num_G', '10_num_T', '11_num_N', '12_num_oth', '13_seq_len']
['chr17', '8148003', '8148983', '3', '0.411224', '0.588776', '213', '288', '289', '190', '0', '0', '980']
['chr17', '8148004', '8148925', '1', '0.412595', '0.587405', '200', '272', '269', '180', '0', '0', '921']
['chr17', '8148004', '8148962', '1', '0.412317', '0.587683', '208', '283', '280', '187', '0', '0', '958']
['chr17', '8148004', '8148963', '1', '0.411887', '0.588113', '208', '284', '280', '187', '0', '0', '959']
['chr17', '8148005', '8149014', '1', '0.412289', '0.587711', '222', '297', '296', '194', '0', '0', '1009']


In [6]:
def prep_line(lst):
    key = "_".join(lst[0:3])
    val = lst[0:3] + lst[4:-1]
    return [key] + val

In [7]:
fdiry = os.path.join(FD_RES, "nuc", "Input1_20x")
fname = "target_PER1.bed.gz"
fpath = os.path.join(fdiry, fname)

n_lines = 5

with gzip.open(fpath, "rb") as file:
    header = file.readline()
    lines  = it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')  
        print(lst)
        print(prep_line(lst))

['chr17', '8148003', '8148983', '3', '0.411224', '0.588776', '213', '288', '289', '190', '0', '0', '980']
['chr17_8148003_8148983', 'chr17', '8148003', '8148983', '0.411224', '0.588776', '213', '288', '289', '190', '0', '0']
['chr17', '8148004', '8148925', '1', '0.412595', '0.587405', '200', '272', '269', '180', '0', '0', '921']
['chr17_8148004_8148925', 'chr17', '8148004', '8148925', '0.412595', '0.587405', '200', '272', '269', '180', '0', '0']
['chr17', '8148004', '8148962', '1', '0.412317', '0.587683', '208', '283', '280', '187', '0', '0', '958']
['chr17_8148004_8148962', 'chr17', '8148004', '8148962', '0.412317', '0.587683', '208', '283', '280', '187', '0', '0']
['chr17', '8148004', '8148963', '1', '0.411887', '0.588113', '208', '284', '280', '187', '0', '0', '959']
['chr17_8148004_8148963', 'chr17', '8148004', '8148963', '0.411887', '0.588113', '208', '284', '280', '187', '0', '0']
['chr17', '8148005', '8149014', '1', '0.412289', '0.587711', '222', '297', '296', '194', '0', '0', '

In [8]:
query_reset = ("DROP TABLE IF EXISTS Fragment")

query_table = ("""CREATE TABLE IF NOT EXISTS Fragment(
    fragment TEXT PRIMARY KEY, 
    chrom    TEXT,
    start    INTEGER,
    end      INTEGER,
    pct_at   REAL,
    pct_gc   REAL,
    num_A    INTEGER,
    num_C    INTEGER,
    num_G    INTEGER,
    num_T    INTEGER,
    num_N    INTEGER,
    num_oth  INTEGER
);""")

query_insert = ("""INSERT OR IGNORE INTO Fragment
    (fragment, chrom, start, end,
     pct_at, pct_gc,
     num_A, num_C, num_G, num_T, num_N, num_oth) 
    VALUES 
    (?,?,?,?,?,?,?,?,?,?,?,?)""")

In [9]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

fdiry = os.path.join(FD_RES, "nuc", "Input1_20x")
fname = "target_PER1.bed.gz"
fpath_gz = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn, gzip.open(fpath_gz, "rb") as file:
    ### reset
    cursor = conn.cursor()
    query  = query_reset
    cursor.execute(query)
    
    ### create table
    query  = query_table
    cursor.execute(query)
    
    ### insert values
    query  = query_insert
    header = file.readline()
    lines  = it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')  
        lst = prep_line(lst)
        cursor.execute(query, lst)
    
    ### show that the table is created
    cursor.execute("SELECT * FROM Fragment")
    for row in cursor.fetchall():
        print(row)

('chr17_8148003_8148983', 'chr17', 8148003, 8148983, 0.411224, 0.588776, 213, 288, 289, 190, 0, 0)
('chr17_8148004_8148925', 'chr17', 8148004, 8148925, 0.412595, 0.587405, 200, 272, 269, 180, 0, 0)
('chr17_8148004_8148962', 'chr17', 8148004, 8148962, 0.412317, 0.587683, 208, 283, 280, 187, 0, 0)
('chr17_8148004_8148963', 'chr17', 8148004, 8148963, 0.411887, 0.588113, 208, 284, 280, 187, 0, 0)
('chr17_8148005_8149014', 'chr17', 8148005, 8149014, 0.412289, 0.587711, 222, 297, 296, 194, 0, 0)


## Insert the whole files

In [10]:
fdiry = os.path.join(FD_RES, "nuc")
fname = "target_PER1.bed.gz"

samples = np.sort(os.listdir(fdiry))

for sam in samples:
    fpath = os.path.join(fdiry, sam, fname)
    print(fpath)

#fpaths = glob.glob(fglob)
#print(fpaths)

/mount/work/out/proj_combeffect/nuc/Input1/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input1_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input2/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input2_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input3/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input3_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input4/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input4_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input5/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/Input5_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/TFX2_DMSO/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/TFX2_Dex/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/TFX3_DMSO/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/TFX3_Dex/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/TFX4_DMSO/target_PER1.bed.gz
/mount/work/out/proj_combeffect/nuc/T

In [11]:
%%time

fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

fdiry = os.path.join(FD_RES, "nuc")
fname = "target_PER1.bed.gz"
samples = np.sort(os.listdir(fdiry))
fpaths = [os.path.join(fdiry, sam, fname) for sam in samples]

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    
    for fpath_gz in fpaths:
        with gzip.open(fpath_gz, "rb") as file:
            ### create table if not exist
            query  = query_table
            cursor.execute(query)

            ### insert values
            query  = query_insert
            header = file.readline()
            lines  = file #it.islice(file, n_lines)
            for line in lines:
                lst = line.decode('ASCII').strip().split('\t')  
                lst = prep_line(lst)
                cursor.execute(query, lst)

CPU times: user 216 ms, sys: 7.09 ms, total: 223 ms
Wall time: 243 ms


**Check**

In [3]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

In [4]:
with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = "select count(*) from Fragment"
    cursor.execute(query)
    print(cursor.fetchall())

[(14694,)]


In [5]:
with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = "SELECT * FROM Fragment"
    cursor.execute(query)
    rows = cursor.fetchall()
    print(len(rows))

14694


In [20]:
%%bash
FDIRY="/mount/work/out/proj_combeffect/nuc"
FD_BEDS=($(ls -d ${FDIRY}/{Input?,Input?_20x,TFX?_DMSO,TFX?_Dex}/))
FN_BED="target_PER1.bed.gz"

sum=0
for FD_BED in ${FD_BEDS[@]}; do
    count=$(zcat ${FD_BED}/${FN_BED} | wc -l)
    echo $count
    sum=$((sum + count))
done
echo
echo $sum

169
3030
191
2934
259
2965
178
3273
128
2813
1720
734
1114
495
1520
510
1832
639

24504


In [22]:
%%bash
FD_BED="/mount/work/out/proj_combeffect/nuc"
FN_BED="target_PER1.bed.gz"

zcat ${FD_BED}/*/${FN_BED} | wc -l

24504


**Check**

In [33]:
fdiry = os.path.join(FD_RES, "nuc")
fname = "target_PER1.bed.gz"
samples = np.sort(os.listdir(fdiry))
fpaths = [os.path.join(fdiry, sam, fname) for sam in samples]

###
lst = list()
for fpath in fpaths:
    dat = pd.read_table(fpath)
    lst.append(dat)
    print(fpath)
    print(dat.shape)

###
dat = pd.concat(lst)
print(dat.shape)

###
tmp = dat.drop(columns="4_usercol").drop_duplicates()
print(tmp.shape)

/mount/work/out/proj_combeffect/nuc/Input1/target_PER1.bed.gz
(168, 13)
/mount/work/out/proj_combeffect/nuc/Input1_20x/target_PER1.bed.gz
(3029, 13)
/mount/work/out/proj_combeffect/nuc/Input2/target_PER1.bed.gz
(190, 13)
/mount/work/out/proj_combeffect/nuc/Input2_20x/target_PER1.bed.gz
(2933, 13)
/mount/work/out/proj_combeffect/nuc/Input3/target_PER1.bed.gz
(258, 13)
/mount/work/out/proj_combeffect/nuc/Input3_20x/target_PER1.bed.gz
(2964, 13)
/mount/work/out/proj_combeffect/nuc/Input4/target_PER1.bed.gz
(177, 13)
/mount/work/out/proj_combeffect/nuc/Input4_20x/target_PER1.bed.gz
(3272, 13)
/mount/work/out/proj_combeffect/nuc/Input5/target_PER1.bed.gz
(127, 13)
/mount/work/out/proj_combeffect/nuc/Input5_20x/target_PER1.bed.gz
(2812, 13)
/mount/work/out/proj_combeffect/nuc/TFX2_DMSO/target_PER1.bed.gz
(733, 13)
/mount/work/out/proj_combeffect/nuc/TFX2_Dex/target_PER1.bed.gz
(1719, 13)
/mount/work/out/proj_combeffect/nuc/TFX3_DMSO/target_PER1.bed.gz
(494, 13)
/mount/work/out/proj_combeffec